In [6]:
import numpy as np
import pandas as pd
import os
import dotenv
import psycopg
from sqlalchemy import create_engine

import dash
from dash import Dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

dotenv.load_dotenv()

True

In [7]:
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

dbms = 'postgresql'
package = 'psycopg'
user = 'postgres'
password = POSTGRES_PASSWORD
host = 'postgres'
port = '5432'
db = 'housing'

engine = create_engine(f'{dbms}+{package}://{user}:{password}@{host}:{port}/{db}')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


In [8]:
acs_annual = pd.read_csv("../data/clean/acs_zillow_fred_long.csv")
zillow_monthly = pd.read_csv("../data/clean/zillow_fred_long.csv")

acs_annual["affordability"] = acs_annual["income"] / acs_annual["hvi"]

msa_list = acs_annual["msa"].sort_values().unique()

In [11]:
msa = "Charlottesville, VA"

df_a = acs_annual[acs_annual["msa"] == msa]
df_m = zillow_monthly[zillow_monthly["msa"] == msa]

fig_income = px.line(df_a, x="year", y="income", title=f"Income Over Time — {msa}")
fig_hvi = px.line(df_a, x="year", y="hvi", title=f"Housing Value Over Time — {msa}")
fig_mort = px.line(df_a, x="year", y="mortgage_rate", title=f"Mortgage Rate Over Time — {msa}")
fig_aff = px.line(df_a, x="year", y="affordability", title=f"Affordability Index — {msa}")

fig_scatter = px.scatter(
df_a, x="income", y="hvi", color="year",
title="Income vs Home Value",
trendline="ols"
)

fig_hvi_m = px.line(df_m, x="month_year", y="hvi",
            title=f"Monthly Housing Value — {msa}")

fig_mort_m = px.line(df_m, x="month_year", y="mortgage_rate",
                title=f"Monthly Mortgage Rate — {msa}")

fig_dual = go.Figure()
fig_dual.add_trace(go.Scatter(
x=df_m["month_year"], y=df_m["hvi"], name="HVI"
))
fig_dual.add_trace(go.Scatter(
x=df_m["month_year"], y=df_m["mortgage_rate"], name="Mortgage Rate", yaxis="y2"
))
fig_dual.update_layout(
title="Monthly HVI vs Mortgage Rate",
yaxis=dict(title="HVI"),
yaxis2=dict(title="Mortgage Rate", overlaying="y", side="right")
)

In [12]:
fig_income.show()

In [13]:
fig_scatter.show()

In [14]:
!pip install statsmodels

In [15]:
acs_annual.dtypes

msa               object
area_name         object
state             object
size_rank          int64
year               int64
income           float64
hvi              float64
mortgage_rate    float64
affordability    float64
dtype: object